In [1]:
# Importing dependencies 
import gmaps
from config import gkey
import pandas as pd
import requests

# Configuring gmaps with my Google API key
gmaps.configure(api_key=gkey)

In [2]:
# Setting the path to the data created in part I and reading it into a dataframe
path = "Resources/city_data.csv"
city_df = pd.read_csv(path)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,provideniya,64.383333,-173.300000,32.27,92.0,37.0,9.44,RU,1.601205e+09
1,fresnillo,23.166667,-102.883333,55.40,66.0,90.0,2.59,MX,1.601206e+09
2,marion,39.770000,-86.160000,61.00,87.0,1.0,9.17,US,1.601206e+09
3,qaanaaq,77.480000,-69.360000,11.05,76.0,1.0,6.91,GL,1.601205e+09
4,auki,-8.766667,160.700000,79.43,81.0,79.0,7.00,SB,1.601206e+09


In [3]:
# Setting the heatmap locations to the latitude and longitude of each city in the dataset
locations = city_df[["Lat", "Lng"]]

# Assigning the weighting of the heatmap to the humidity of the city
rating = city_df["Humidity"].astype(float)

In [4]:
# Use the Gmaps module to load the figure
fig = gmaps.figure()

# Set up the heatmap with the heatmap_layer function
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.1)

# Adding the heatmat layer to the figure 
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Filtering the dataframe to only include weather conditions which are ideal
hotel_df = city_df.loc[(city_df["Max Temp"] > 80) & (city_df["Cloudiness"] < 10) & (city_df["Wind Speed"] < 5), :]

# Removing any NaN values from the dataframe
hotel_df = hotel_df.dropna()

hotel_df["Hotel Name"] = ""

# Checking how many remaining cities there are to choose from and previewing the data
print(f"The number of possible vacation destinations is: {len(hotel_df)}")
hotel_df

The number of possible vacation destinations is: 16


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,kita,13.030000,-9.490000,86.58,58.0,0.0,4.34,ML,1.601206e+09,
209,kalabo,-14.966667,22.683333,102.92,8.0,9.0,4.32,ZM,1.601205e+09,
260,bulawayo,-20.150000,28.583333,95.56,10.0,0.0,4.03,ZW,1.601206e+09,
261,chissamba,-12.166667,17.333333,98.91,16.0,2.0,2.01,AO,1.601206e+09,
280,shakawe,-18.366667,21.850000,101.57,8.0,0.0,4.03,BW,1.601206e+09,
303,venice,27.100000,-82.450000,81.00,88.0,1.0,4.70,US,1.601206e+09,
355,kasane,-17.816667,25.150000,98.60,6.0,0.0,4.70,BW,1.601206e+09,
427,alexandria,31.220000,29.960000,91.40,41.0,0.0,4.70,EG,1.601206e+09,
473,guiratinga,-16.350000,-53.760000,82.20,48.0,0.0,4.68,BR,1.601206e+09,
525,itapuranga,-15.560000,-49.950000,80.51,30.0,0.0,2.51,BR,1.601206e+09,


In [6]:
# Creating the API call to find the nearest hotel within 5000m of the ideal cities
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    params = {
        "location": f"{row[1]}, {row[2]}",
        "keyword": "Hotel",
        "radius": 5000,
        "key": gkey,
    }
    
    city_name = row["City"]
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("------------")

Retrieving Results for Index 24: kita.
Closest hotel to kita is Hotel Nelson Mandela.
------------
Retrieving Results for Index 209: kalabo.
Closest hotel to kalabo is HM Sams Lodge.
------------
Retrieving Results for Index 260: bulawayo.
Closest hotel to bulawayo is Holiday Inn Bulawayo.
------------
Retrieving Results for Index 261: chissamba.
Missing field/result... skipping.
------------
Retrieving Results for Index 280: shakawe.
Closest hotel to shakawe is Athoma Guest House.
------------
Retrieving Results for Index 303: venice.
Closest hotel to venice is Gulf Shore Beach Resort.
------------
Retrieving Results for Index 355: kasane.
Closest hotel to kasane is Chobe Water Villas.
------------
Retrieving Results for Index 427: alexandria.
Closest hotel to alexandria is Four Seasons Hotel Alexandria at San Stefano, Egypt.
------------
Retrieving Results for Index 473: guiratinga.
Closest hotel to guiratinga is Hotel Angra do Cerrado - Guiratinga MT.
------------
Retrieving Results

In [7]:
# For the rows where the hotel data was not found, drop those rows from the dataframe
NaN = float("NaN")
hotel_df.replace("", NaN, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

# Display the final dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,kita,13.030000,-9.490000,86.58,58.0,0.0,4.34,ML,1.601206e+09,Hotel Nelson Mandela
209,kalabo,-14.966667,22.683333,102.92,8.0,9.0,4.32,ZM,1.601205e+09,HM Sams Lodge
260,bulawayo,-20.150000,28.583333,95.56,10.0,0.0,4.03,ZW,1.601206e+09,Holiday Inn Bulawayo
280,shakawe,-18.366667,21.850000,101.57,8.0,0.0,4.03,BW,1.601206e+09,Athoma Guest House
303,venice,27.100000,-82.450000,81.00,88.0,1.0,4.70,US,1.601206e+09,Gulf Shore Beach Resort
355,kasane,-17.816667,25.150000,98.60,6.0,0.0,4.70,BW,1.601206e+09,Chobe Water Villas
427,alexandria,31.220000,29.960000,91.40,41.0,0.0,4.70,EG,1.601206e+09,"Four Seasons Hotel Alexandria at San Stefano, ..."
473,guiratinga,-16.350000,-53.760000,82.20,48.0,0.0,4.68,BR,1.601206e+09,Hotel Angra do Cerrado - Guiratinga MT
525,itapuranga,-15.560000,-49.950000,80.51,30.0,0.0,2.51,BR,1.601206e+09,Hotel Alabama
562,nsanje,-16.916667,35.260000,114.46,9.0,1.0,3.18,MW,1.601206e+09,Shumba Lodge


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Create the symbol layer
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)

# Use the Gmaps module to load the figure
fig = gmaps.figure()

# Adding the heatmap layer and marker layer
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))